<a href="https://colab.research.google.com/github/trvoid/llm-study/blob/main/distilbert/distilbert_pretraining_for_won.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT 사전 학습

## 토크나이저 훈련

In [ ]:
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

corpus_file = "won/won04-gyojeon.txt"
data_files = [
    "won/won04-gyojeon.txt",
    "won/won05-gyosa.txt"
]

output_dir = "won/tokenizers"
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

vocab_sizes = [5000, 10000]
limit_alphabet = 6000
min_frequency = 5
model_names = ["bpe", "unigram", "wordpiece"]

for vocab_size in vocab_sizes:
    for model_name in model_names:
        if model_name == "bpe":
            tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
            trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                                vocab_size=vocab_size,
                                limit_alphabet=limit_alphabet,
                                min_frequency=min_frequency)
        elif model_name == "unigram":
            tokenizer = Tokenizer(Unigram())
            trainer = UnigramTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                                unk_token="[UNK]",
                                vocab_size=vocab_size,
                                limit_alphabet=limit_alphabet,
                                min_frequency=min_frequency)
        elif model_name == "wordpiece":
            tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
            trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                                vocab_size=vocab_size,
                                limit_alphabet=limit_alphabet,
                                min_frequency=min_frequency)

        tokenizer.pre_tokenizer = Whitespace()

        tokenizer.train(data_files, trainer)

        tokenizer_dir = os.path.join(output_dir, f"{model_name}_{vocab_size}")
        if not os.path.exists(tokenizer_dir):
            os.makedirs(tokenizer_dir, exist_ok=True)
        tokenizer_file = os.path.join(tokenizer_dir, "tokenizer.json")
        tokenizer.save(tokenizer_file)
        print(f"Saved tokenizer: {tokenizer_file}")


Saved tokenizer: won/tokenizers/bpe_5000/tokenizer.json
Saved tokenizer: won/tokenizers/unigram_5000/tokenizer.json
Saved tokenizer: won/tokenizers/wordpiece_5000/tokenizer.json
Saved tokenizer: won/tokenizers/bpe_10000/tokenizer.json
Saved tokenizer: won/tokenizers/unigram_10000/tokenizer.json
Saved tokenizer: won/tokenizers/wordpiece_10000/tokenizer.json


## 토크나이저 사용

In [ ]:
texts = [
    "물질이 개벽되니 정신을 개벽하자",
    "19.대종사 말씀하시기를 [스승이 법을 새로 내는 일이나"
]

for text in texts:
    print("#" * 80)
    print("TEXT: " + text)
    print("-" * 80)
    for vocab_size in vocab_sizes:
        for model_name in model_names:
            try:
                tokenizer_file = os.path.join(output_dir, f"{model_name}_{vocab_size}", "tokenizer.json")
                tokenizer = Tokenizer.from_file(tokenizer_file)
                print(f"{model_name:9} {vocab_size:5}: {tokenizer.encode(text).tokens}")
            except Exception as e:
                print(f"{model_name:9} {vocab_size:5}: FAIL")
                print(e)

################################################################################
TEXT: 물질이 개벽되니 정신을 개벽하자
--------------------------------------------------------------------------------
bpe        5000: ['물질', '이', '개벽', '되니', '정신을', '개벽', '하자']
unigram    5000: ['물질', '이', '개벽', '되니', '정신', '을', '개벽', '하자']
wordpiece  5000: ['물질', '##이', '개', '##벽', '##되', '##니', '정신을', '개', '##벽', '##하', '##자']
bpe       10000: ['물질이', '개벽', '되니', '정신을', '개벽', '하자']
unigram   10000: ['물질이', '개벽', '되니', '정신', '을', '개벽', '하자']
wordpiece 10000: ['물질이', '개벽', '##되니', '정신을', '개벽', '##하자']
################################################################################
TEXT: 19.대종사 말씀하시기를 [스승이 법을 새로 내는 일이나
--------------------------------------------------------------------------------
bpe        5000: ['19', '.', '대종사', '말씀하시기를', '[', '스승이', '법을', '새로', '내는', '일이나']
unigram    5000: ['19', '.', '대종사', '말씀하시', '기', '를', '[', '스승이', '법', '을', '새로', '내', '는', '일이', '나']
wordpiece  5000: ['19', '.', '대종사', '말

## 데이터셋 준비

In [ ]:
from transformers import (
    PreTrainedTokenizerFast,
    LineByLineTextDataset,
    DataCollatorForLanguageModeling
)

tokenizer_dir = os.path.join(output_dir, "unigram_5000")
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    tokenizer_dir,
    unk_token="[UNK]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token="[PAD]",
    mask_token="[MASK]"
)
print(tokenizer.vocab_size)
#tokenizer = Tokenizer.from_file("./tokenizer_5000_wordpiece/tokenizer.json")

5000


In [ ]:
# MLM을 위한 데이터셋
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=corpus_file,
    block_size=128  # 토큰 기준 최대 길이
)

# MLM 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## 모델 구성

In [ ]:
from transformers import DistilBertConfig, DistilBertForMaskedLM

#dim = 768                 # Hidden size
#hidden_dim = 3072         # Intermediate size (dim * 4)
#n_layers = 6              # Number of layers
#n_heads = 12              # Number of attention heads
#max_position_embeddings = 512

dim = 60
hidden_dim = 128
n_layers = 3
n_heads = 6
max_position_embeddings = 256

# 표준 DistilBERT-base 구성과 유사하게 설정
config = DistilBertConfig(
    vocab_size=tokenizer.vocab_size, # 로드한 토크나이저의 어휘 크기 사용
    activation="gelu",
    dim=dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    n_heads=n_heads,
    max_position_embeddings=max_position_embeddings
    # dropout, attention_dropout 등 다른 파라미터도 설정 가능
)

# 모델 초기화 (랜덤 가중치)
model = DistilBertForMaskedLM(config=config)
print(model.num_parameters())

415544


## 모델 훈련

In [ ]:
from transformers import TrainingArguments, Trainer

os.environ["WANDB_DISABLED"] = "true"

# 학습 인자 설정
output_dir = "./my_distilbert_pretrained_mlm"
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=40,       # 실제로는 훨씬 더 많이 필요
    per_device_train_batch_size=8, # GPU 메모리에 맞춰 조절
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True, # 가능하면 True
    # learning_rate, weight_decay 등 추가 설정 필요
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# --- 6. 모델 학습 ---
print("DistilBERT 구조 모델 사전 학습(MLM)을 시작합니다. (매우 간소화된 예시)")
trainer.train()
print("사전 학습 완료.")

# --- 7. 모델 저장 ---
print(f"학습된 모델과 토크나이저를 {output_dir}에 저장합니다.")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir) # 사용한 토크나이저 함께 저장

print("저장 완료.")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


DistilBERT 구조 모델 사전 학습(MLM)을 시작합니다. (매우 간소화된 예시)


Step,Training Loss
500,8.002300
1000,7.142500
1500,6.718400
2000,6.549400
2500,6.468000
3000,6.455900
3500,6.437200
4000,6.450000
4500,6.422800
5000,6.415400


사전 학습 완료.
학습된 모델과 토크나이저를 ./my_distilbert_pretrained_mlm에 저장합니다.
저장 완료.


## 모델 사용

In [ ]:
# 사용 예시 (저장된 모델 로드)
from transformers import AutoTokenizer, DistilBertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = DistilBertForMaskedLM.from_pretrained(output_dir)

print(len(tokenizer))

5000


In [ ]:
!pip install datasets

In [ ]:
import torch
from datasets import Dataset

#device = "cpu"

def find_topk_for_masked(tokenizer, model, text, topk=5):
    inputs = tokenizer(text, return_tensors="pt")
    #inputs = {k: v.to(device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')

    token_logits = model(**inputs).logits
    print(token_logits.shape)

    # [MASK]의 위치를 찾고, 해당 logits을 추출합니다.
    #print(torch.where(inputs["input_ids"] == tokenizer.mask_token_id))
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    #print(mask_token_index)
    mask_token_logits = token_logits[0, mask_token_index, :]
    #print(mask_token_logits)

    # 가장 큰 logits값을 가지는 [MASK] 후보를 선택합니다.
    top_5_tokens = torch.topk(mask_token_logits, topk, dim=1).indices[0].tolist()

    return top_5_tokens

In [ ]:
test_texts = [
    "물질이 개벽되니 [MASK]을 개벽하자",
    "19.대종사 말씀하시기를 [스승이 [MASK]을 새로 내는 일이나",
    "19.대종사 [MASK]기를"
]

for text in test_texts:
    print(f"'input text: {text}'")
    try:
        topk_tokens = find_topk_for_masked(tokenizer, model, text, topk=5)
        for token in topk_tokens:
            print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")
    except Exception as e:
        print(f"Exception: {e}")

'input text: 물질이 개벽되니 [MASK]을 개벽하자'
torch.Size([1, 8, 5000])
'>>> 물질이 개벽되니 을을 개벽하자'
'>>> 물질이 개벽되니 .을 개벽하자'
'>>> 물질이 개벽되니 이을 개벽하자'
'>>> 물질이 개벽되니 ,을 개벽하자'
'>>> 물질이 개벽되니 를을 개벽하자'
'input text: 19.대종사 말씀하시기를 [스승이 [MASK]을 새로 내는 일이나'
torch.Size([1, 15, 5000])
'>>> 19.대종사 말씀하시기를 [스승이 을을 새로 내는 일이나'
'>>> 19.대종사 말씀하시기를 [스승이 ,을 새로 내는 일이나'
'>>> 19.대종사 말씀하시기를 [스승이 를을 새로 내는 일이나'
'>>> 19.대종사 말씀하시기를 [스승이 이을 새로 내는 일이나'
'>>> 19.대종사 말씀하시기를 [스승이 .을 새로 내는 일이나'
'input text: 19.대종사 [MASK]기를'
torch.Size([1, 6, 5000])
'>>> 19.대종사 을기를'
'>>> 19.대종사 ,기를'
'>>> 19.대종사 이기를'
'>>> 19.대종사 .기를'
'>>> 19.대종사 를기를'
